# 5.Основы парсинга и работы с API

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Задание 1.
<b>Обязательная часть</b>

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

<i>KEYWORDS = [‘python’, ‘парсинг’]</i>

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

<b>Дополнительная часть (необязательная)</b>

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [ ]:
request = requests.get('https://habr.com/ru/all/')

request.text

In [ ]:
soup = BeautifulSoup(request.text)
soup

In [ ]:
def add_full_text(link):
  #print(link)
  headers = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
  }
  url = link
  req = requests.get(url = url, headers = headers).text
  soup = BeautifulSoup(req)
  full_ = soup.find('div', {"id":"post-content-body"}).text
  return full_

def find_posts():
  url = 'https://habr.com/ru/all/'
  keywords = ['python', 'парсинг']
  headers = {
      'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
  }
  req = requests.get(url = url, headers = headers).text
  soup = BeautifulSoup(req)

  news = soup.find_all('article', {"class": "tm-articles-list__item"})
  #print(news[0])

  habr_blog = pd.DataFrame()

  for el in news:
    date = el.find('span', class_= 'tm-article-snippet__datetime-published').find('time').get('datetime')
    title = el.find('h2').text
    link = el.find('h2').find('a').get('href')
    text = add_full_text('https://habr.com' + link)
    row = {'date': date, 'title': title, 'link': link, 'text': text}

    for s in keywords:
      if s.lower() in text.lower():
        habr_blog = pd.concat([habr_blog, pd.DataFrame([row])])
        break

  return habr_blog.reset_index(drop=True)
  
find_posts()

,date,title,link,text
0,2021-11-23T09:07:17.000Z,EMM Всемогущий: как подчинить 170 тысяч корпор...,/ru/company/magnit/blog/590929/,"Всем привет! Меня зовут Илья Никулин, с марта ..."
1,2021-11-23T08:47:57.000Z,Представляем Waypoint от HashiCorp,/ru/company/nixys/blog/588793/,"\nУстойчивый рабочий процесс для сборки, депло..."
2,2021-11-23T08:38:43.000Z,Искусственный интеллект лицом,/ru/post/588580/,или веб морда для ваших поделок ( Пет проект )...


In [ ]:
result = find_posts()

result['text'][0]

### Задание 2.

<b>Обязательная часть</b>

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check. Список email-ов задаем переменной в начале кода:

<i>EMAIL = [xxx@x.ru, yyy@y.com]</i>

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

<b><small>Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.</small></b>

<b>Дополнительная часть (необязательная)</b>

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.

<b><small>Документация к API VK: https://vk.com/dev/methods, вам поможет метод wall.get</small></b>

<i>GROUP = 'netology’</i>

<i>TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ</i>

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [ ]:
def leaks():
  url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
  payload = {"emailAddresses": ['xxx@x.ru', 'yyy@y.com']}
  headers = {
      'Vaar-Header-App-Build-Version': '1.0.0',
      'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
      'Vaar-Version': '0'
      }
  req = requests.post(url = url, json = payload, headers = headers)
  response = req.json()

  df = pd.DataFrame(req.json()['breaches']).T
  return df

leaks()

,breachId,site,recordsCount,description,publishDate,statistics
17110,17110,azcentral.com,705538,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z,"{'usernames': 0, 'passwords': 702971, 'emails'..."
37177,37177,forums.vkmonline.com,825654,"At an unconfirmed date, the Russian-language m...",2021-02-11T00:00:00Z,"{'usernames': 825566, 'passwords': 825654, 'em..."
41,41,dropbox.com,68591031,Cloud storage company Dropbox suffered a major...,2016-10-24T00:00:00Z,"{'usernames': 0, 'passwords': 68591031, 'email..."
17009,17009,zynga.com,216159495,"In September 2019, the game developer Zynga wa...",2019-10-17T00:00:00Z,"{'usernames': 214648143, 'passwords': 13476279..."
12,12,vk.com,110121799,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z,"{'usernames': 0, 'passwords': 107425364, 'emai..."
3,3,adobe.com,152046506,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z,"{'usernames': 0, 'passwords': 129430596, 'emai..."
13662,13662,netlog.com,52904632,Netlog (formerly known as Facebox and Bingbox)...,2018-02-18T00:00:00Z,"{'usernames': 0, 'passwords': 52902665, 'email..."
3520,3520,globalreach.eu,921298,"In 2016, Global Reach Technology's database wa...",2017-03-15T00:00:00Z,"{'usernames': 13492, 'passwords': 921298, 'ema..."
3164,3164,cfire.mail.ru,12880232,"In July and August of 2016, two criminals carr...",2017-02-14T00:00:00Z,"{'usernames': 12880232, 'passwords': 12880222,..."
2961,2961,cdprojektred.com,1871459,"In March 2016, CDProjektRed.com.com's forum da...",2017-01-31T00:00:00Z,"{'usernames': 1871410, 'passwords': 1871457, '..."


In [ ]:
data = leaks()

In [ ]:
res = data.reset_index()

In [ ]:
res[['publishDate', 'site', 'description']]

,publishDate,site,description
0,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
1,2021-02-11T00:00:00Z,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."
2,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
3,2019-10-17T00:00:00Z,zynga.com,"In September 2019, the game developer Zynga wa..."
4,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
5,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
6,2018-02-18T00:00:00Z,netlog.com,Netlog (formerly known as Facebox and Bingbox)...
7,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
8,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
9,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
